# Sub Question Query Engine
In this tutorial, we showcase how to use a **sub question query engine** to tackle the problem of answering a complex query using multiple data sources.  
It first breaks down the complex query into sub questions for each relevant data source,
then gather all the intermediate reponses and synthesizes a final response.

### Preparation

In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index import ServiceContext

In [2]:
# load data
pg_essay = SimpleDirectoryReader(input_dir="../data/paul_graham/").load_data()

# build index and query engine
query_engine = VectorStoreIndex.from_documents(pg_essay).as_query_engine()

### Setup sub question query engine

In [3]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine, 
        metadata=ToolMetadata(name='pg_essay', description='Paul Graham essay on What I Worked On')
    )
]

# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])
service_context = ServiceContext.from_defaults(callback_manager=callback_manager)
query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools,
                                                    service_context=service_context,
                                                    use_async=False)

### Run queries

In [4]:
response = await query_engine.aquery('How was Paul Grahams life different before and after YC?')

Generated 2 sub questions.
[pg_essay] Q: What did Paul Graham work on before YC?
[pg_essay] Q: What did Paul Graham work on after YC?
[pg_essay] A: 
Before YC, Paul Graham worked on hacking, writing essays, and Arc, a programming language. He also wrote Hacker News, originally called Startup News, to test the new version of Arc.
[pg_essay] A: 
After YC, Paul Graham worked on painting, writing essays, and Lisp.
**********
Trace: query
    |_llm ->  5.140319 seconds
    |_sub_questions ->  4.561899 seconds
    |_synthesize ->  2.904701 seconds
      |_llm ->  2.902163 seconds
**********


In [5]:
print(response)


Paul Graham's life was significantly different before and after YC. Before YC, he was focused on hacking, writing essays, and developing a programming language. After YC, he shifted his focus to painting, writing essays, and Lisp.


In [6]:
# iterate through sub_question items captured in SUB_QUESTIONS event
from llama_index.callbacks.schema import CBEventType, EventPayload

for start_event, end_event in llama_debug.get_event_pairs(CBEventType.SUB_QUESTIONS):
    print(end_event.payload)
    for i, qa_pair in enumerate(end_event.payload[EventPayload.SUB_QUESTIONS]):
        print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
        print("Answer: " + qa_pair.answer.strip())
        print("====================================")

{<EventPayload.SUB_QUESTIONS: 'sub_questions'>: [SubQuestionAnswerPair(sub_q=SubQuestion(sub_question='What did Paul Graham work on before YC?', tool_name='pg_essay'), answer='\nBefore YC, Paul Graham worked on hacking, writing essays, and Arc, a programming language. He also wrote Hacker News, originally called Startup News, to test the new version of Arc.'), SubQuestionAnswerPair(sub_q=SubQuestion(sub_question='What did Paul Graham work on after YC?', tool_name='pg_essay'), answer='\nAfter YC, Paul Graham worked on painting, writing essays, and Lisp.')]}
Sub Question 0: What did Paul Graham work on before YC?
Answer: Before YC, Paul Graham worked on hacking, writing essays, and Arc, a programming language. He also wrote Hacker News, originally called Startup News, to test the new version of Arc.
Sub Question 1: What did Paul Graham work on after YC?
Answer: After YC, Paul Graham worked on painting, writing essays, and Lisp.
